In [2]:
import datetime
import logging
import numpy as np
import os
from typing import Optional

import pandas as pd
import torch
import tqdm
from sklearn.model_selection import train_test_split
from tensorboardX import SummaryWriter
from torch import nn, optim
from torch.utils.data import DataLoader

# from models.custom_models.gtn import GatedTransformerNetwork
from models.gtn.transformer import Transformer
from utils.config import gtn_param
from utils.loader import make_loader
from utils.path_utils import project_root
from utils.plot_metrics import plot_losses_and_accuracies

from utils.config import masked_gtn_param
from models.custom_models.gtn_mask import MaskedGatedTransformerNetwork
device = 'cuda'
config = gtn_param


In [3]:
device = 'cuda'
config = masked_gtn_param
d_input, d_channel, d_output = 336, 191, 2

model = MaskedGatedTransformerNetwork(d_model=config['d_model'], d_input=d_input, d_channel=d_channel,
                                          d_output=d_output, d_hidden=config['d_hidden'], q=config['q'],
                                          v=config['v'], h=config['h'], N=config['N'], dropout=config['dropout'],
                                          pe=config['pe'], mask=config['mask'], device=device).to(device)

In [7]:
param_size = 0

for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 193.365MB


# Hyperparameter Tuning

In [21]:
def power(a, b):
    return a**b

from functools import partial, partialmethod

a_square = partial(power, b=2)
b_square = partialmethod(power, a=2)

In [24]:
b_square

functools.partialmethod(<function power at 0x78e8ae250900>, , a=2)

# Cross Validation

In [32]:
from train_gtn import initialize_experiment

data_file = "final_dataset.pickle"
training_examples, lengths_list, is_sepsis, writer, destination_path = initialize_experiment(data_file)

sepsis = pd.Series(is_sepsis)
positive_sepsis_idxs = sepsis[sepsis == 1].index
negative_sepsis_idxs = sepsis[sepsis == 0].sample(frac=0.20).index
all_samples = list(positive_sepsis_idxs) + list(negative_sepsis_idxs)
np.random.shuffle(all_samples)

print(f"Number of positive samples: {len(positive_sepsis_idxs)}")
print(f"Number of negative samples: {len(negative_sepsis_idxs)}")

# Reducing the samples to have balanced dataset
batch_size = config['batch_size'] * torch.cuda.device_count()
print(f"Batch size: {batch_size}")
logging.info(f"Batch size: {batch_size}")

# Splitting dataset into train and test
print(f"Total samples: {len(all_samples)}")

train_indicies, temp_indicies = train_test_split(all_samples, test_size=0.2, random_state=42)  # 80 20
val_indicies, test_indicies = train_test_split(temp_indicies, test_size=0.5, random_state=42)  # 10 10

# train_indicies, test_indicies = train_test_split(all_samples, test_size=0.2, random_state=42)
# train_loader, test_loader, train_indicies, test_indicies = make_loader(training_examples, lengths_list, is_sepsis,
#                                                                        batch_size=batch_size, mode='padding',
#                                                                        num_workers=4, train_indicies=train_indicies,
#                                                                        test_indicies=test_indicies, include_val=True)

train_loader, val_loader, test_loader, train_indicies, val_indices, test_indicies = make_loader(
    training_examples, lengths_list, is_sepsis, batch_size=batch_size, mode='padding', num_workers=4,
    train_indicies=train_indicies, test_indicies=test_indicies, val_indicies=val_indicies,
    select_important_features=False, include_val=False)

Datafile used: final_dataset.pickle
Total number of patients: 40336
Min recordings: 8 & Max recordings: 336
Distribution of the SepsisLabel: 
0    37404
1     2932
Name: count, dtype: int64
Number of positive samples: 2932
Number of negative samples: 7481
Batch size: 3
Total samples: 10413
Loading train and test from given indicies


Padding...: 100%|██████████| 1042/1042 [00:03<00:00, 299.42it/s]


ValueError: not enough values to unpack (expected 6, got 4)

In [28]:
from torch.utils.data import TensorDataset
from sklearn.model_selection import KFold

data = torch.stack([train_loader.dataset[i][0] for i in range(len(train_loader.dataset))])
labels = torch.tensor([train_loader.dataset[i][1] for i in range(len(train_loader.dataset))])

kfold = KFold(n_splits=5, random_state=42, shuffle=True)

for train_index, test_index in kfold.split(data):
    
    train_index = torch.tensor(train_index)
    test_index = torch.tensor(test_index)
    
    X_train_fold = data[train_index]
    y_train_fold = labels[train_index]
    X_test_fold = data[test_index]
    y_test_fold = labels[test_index]
    
    train_loader = DataLoader(TensorDataset(X_train_fold, y_train_fold), batch_size=3, shuffle=True)
    test_loader = DataLoader(TensorDataset(X_test_fold, y_test_fold), batch_size=3, shuffle=False)
    

# TARNet

In [113]:
# from models.tarnet.multitask_transformer_class import MultitaskTransformerModel
# from train_tarnet import load_model
# from utils.config import tarnet_param


In [114]:
# config = tarnet_param
# (d_input, d_channel), d_output = (336, 191), 2
# 
# model = MultitaskTransformerModel(task_type=config['task_type'], device=config['device'],
#                                       nclasses=d_output, seq_len=d_input, batch=16,
#                                       input_size=d_channel, emb_size=config['emb_size'],
#                                       nhead=config['nhead'], nhid=config['nhid'], nhid_tar=config['nhid_tar'],
#                                       nhid_task=config['nhid_task'], nlayers=config['nlayers'],
#                                       dropout=config['dropout'], )

In [115]:
# load_model(model, model_name='./saved_models/tarnet/tarnet_final_2_val.pkl')

In [116]:
# from train_tarnet import initialize_experiment
# 
# # Getting Data and Loaders
# data_file = "final_dataset.pickle"
# training_examples, lengths_list, is_sepsis, writer, destination_path = initialize_experiment(data_file)
# 
# sepsis = pd.Series(is_sepsis)
# positive_sepsis_idxs = sepsis[sepsis == 1].index
# negative_sepsis_idxs = sepsis[sepsis == 0].sample(frac=0.20).index
# all_samples = list(positive_sepsis_idxs) + list(negative_sepsis_idxs)
# np.random.shuffle(all_samples)
# 
# logging.info(f"Number of positive samples: {len(positive_sepsis_idxs)}")
# logging.info(f"Number of negative samples: {len(negative_sepsis_idxs)}")
# 
# # Reducing the samples to have balanced dataset
# batch_size = config['batch'] * torch.cuda.device_count()
# print(f"Batch size: {batch_size}")
# logging.info(f"Batch size: {batch_size}")
# 
# # Splitting dataset into train and test
# logging.info(f"Total samples: {len(all_samples)}")
# 
# train_indicies, temp_indicies = train_test_split(all_samples, test_size=0.2, random_state=42)  # 80 20
# val_indicies, test_indicies = train_test_split(temp_indicies, test_size=0.5, random_state=42)  # 10 10
# 
# train_loader, val_loader, test_loader, train_indicies, val_indices, test_indicies = make_loader(
#     training_examples, lengths_list, is_sepsis, batch_size=1, mode='padding', num_workers=4,
#     train_indicies=train_indicies, test_indicies=test_indicies, val_indicies=val_indicies,
#     select_important_features=False, include_val=True)

In [117]:
# import math
# import torch.nn.functional as F
# 
# targets_list = []
# predictions = []
# probas = []
# 
# def make_perfect_batch(X, num_inst, num_samples):
#     extension = np.zeros((num_samples - num_inst, X.shape[1], X.shape[2]))
#     X = np.concatenate((X, extension), axis=0)
#     return X
# 
# def tarnet_preprocessing(patient_data):
#     # print(patient_data.shape, type(patient_data))
#     patient_data = patient_data.detach().cpu().numpy()
#     # print(patient_data.shape, type(patient_data))
#     num_train_inst = patient_data.shape[0]
#     num_train_samples = math.ceil(num_train_inst / 16) * 16  # Original model batch_size was 16
#     patient_data = make_perfect_batch(patient_data, num_train_inst, num_train_samples)
#     # print(patient_data.shape, type(patient_data))
#     
#     return torch.as_tensor(patient_data).float()
# 
# for patient_data, targets in tqdm.tqdm(val_loader):
#     patient_data = tarnet_preprocessing(patient_data)
#     with torch.no_grad():
#         patient_data = patient_data.to(device)
#         outputs, _ = model(patient_data, task_type='classification')
#         # outputs = outputs[0]
#         _, predicted = torch.max(outputs, 1)
#         probabilities = F.softmax(outputs, dim=1)
#     
#         predicted_class = predicted.detach().cpu().numpy()[0]
# 
#         predictions.append(predicted_class)
#         targets_list.append(targets.detach().cpu().numpy()[0])
#         probas.append(probabilities.detach().cpu().numpy()[0][predicted_class])